In [1]:
import numpy as np
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

from mcda.matrices import PerformanceTable
from mcda.scales import QuantitativeScale, PreferenceDirection
from mcda.functions import VShapeFunction
from mcda.outranking.promethee import Promethee1

from methods.promethee_complete import promethee_complete_pulp
from methods.promethee_partial import promethee_partial_pulp

In [2]:
labels = ["A1", "A2", "A3", "A4", "A5", "A6", "A7"]

weights = [8, 4, 7, 2, 5]

# Definicja alternatyw
alternatives = [
    "A1",
    "A2",
    "A3",
    "A4",
    "A5",
    "A6",
    "A7"
]

# Definicja skali dla każdego kryterium
scales = {
    0: QuantitativeScale(1000, 4000, preference_direction=PreferenceDirection.MIN),# Price 
    1: QuantitativeScale(32, 1024, preference_direction=PreferenceDirection.MAX),    # Memory Size
    2: QuantitativeScale(8, 120, preference_direction=PreferenceDirection.MAX),     # Camera 
    3: QuantitativeScale(0, 5, preference_direction=PreferenceDirection.MAX),      # Functionalities 
    4: QuantitativeScale(12, 32, preference_direction=PreferenceDirection.MAX),      # Battery life
}

dataset = PerformanceTable(
    [[2200, 256, 32, 2, 24], [3200, 128, 64, 2, 18], [4000, 128, 32, 3, 22], [2400, 64, 32, 2, 28], [1400, 256, 120, 4, 14], [1600, 128, 120, 3, 14], [3200, 512, 16, 2, 20]],
    alternatives=alternatives, scales=scales
)
# Wagi kryteriów
W = {
    0: weights[0],  # Price 
    1: weights[1],   # Memory Size
    2: weights[2],  # Camera 
    3: weights[3],  # Functionalities 
    4: weights[4]   # Battery life
}



preference_func_list = {0: VShapeFunction(p=600, q=300),
                        1: VShapeFunction(p=128, q=64),
                        2: VShapeFunction(p=32, q=12),
                        3: VShapeFunction(p=3, q=2),
                        4: VShapeFunction(p=6, q=2)}

In [3]:
promethee1 = Promethee1(dataset, W, preference_func_list)

In [5]:
promethee_matrix = promethee1.preferences(promethee1.partial_preferences()).data.values

In [7]:
print(promethee_matrix.round(2))

[[0.   0.65 0.46 0.15 0.19 0.35 0.46]
 [0.27 0.   0.58 0.27 0.1  0.1  0.27]
 [0.   0.1  0.   0.   0.19 0.19 0.05]
 [0.1  0.5  0.5  0.   0.19 0.19 0.55]
 [0.58 0.73 0.73 0.73 0.   0.15 0.58]
 [0.58 0.58 0.58 0.58 0.   0.   0.58]
 [0.15 0.15 0.46 0.15 0.35 0.35 0.  ]]


In [8]:
r_pc, z_pc, obj_pc = promethee_complete_pulp(promethee_matrix)

print("PROMETHEE COMPLETE\n")
print(r_pc)
print(obj_pc)

PROMETHEE COMPLETE

[[0 1 1 1 0 0 1]
 [0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0]
 [1 1 1 0 0 0 1]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 1]
 [0 1 1 0 0 0 0]]
23.23846153846154


In [9]:
r_pp, Pp_pp, Pm_pp, I_pp, R_pp, obj_pp = promethee_partial_pulp(promethee_matrix)

print("PROMETHEE PARTIAL\n")
print(r_pp)
print(obj_pp)

PROMETHEE PARTIAL

[[0 1 1 1 0 0 1]
 [0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0]
 [1 1 1 0 0 0 1]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 1]
 [0 1 1 0 0 0 0]]
23.23846153846154
